# The PolInSAR Course - May 27, 2024
# SAR Polarimetry (PolSAR) 
# Part 1: The scattering matrix

**Input data:**

- Acquisition: Nkok (Gabon), DLR's F-SAR, L-band

- Path to images: /projects/data/polsar/

- SLC (single-look complex) images:
    - HH: slc_16afrisr0107_Lhh_tcal_test.rat
    - HV: slc_16afrisr0107_Lhv_tcal_test.rat
    - VH: slc_16afrisr0107_Lvh_tcal_test.rat
    - VV: slc_16afrisr0107_Lvh_tcal_test.rat


In [ ]:
# --- Download exercise data & import reader function
from pysarpro import io, data
from pysarpro.io import rrat

data.download_all(directory="/projects", pattern=r'^data/polsar')

# --- Import useful libaries, functions, and modules
import sys
sys.path.append('/projects/src/')
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import uniform_filter
%matplotlib widget

**Auxiliary functions**

In [ ]:
def HSV_colormap_to_rgb(colormap, h, s, v):
    """
    Makes an HSV-like RGB representation based on the given colormap instead
    of 'hsv' colormap.
    
    See https://en.wikipedia.org/wiki/HSL_and_HSV

    Parameters
    ----------
    colormap : function
        Colormap function. Takes the values in 'h' array and returns an RGBA
        value for each point. The ones in matplotlib.cm should be compatible
    h : ndarray
        Hue values. Usually between 0 and 1.0.
    s : ndarray
        Saturation values. Between 0 and 1.0.
    v : ndarray
        Value values. Between 0 and 1.0.

    Returns
    -------
    rgb: ndarray
        An array with the same shape as input + (3,) representing the RGB.
    """
    # Generate color between given colormap (colormap(h)) and white (ones)
    # according to the given saturation
    tmp = (1-s)[..., np.newaxis]*np.ones(3) + s[..., np.newaxis] * colormap(h)[...,:3]
    # Scale it by value
    return v[..., np.newaxis] * tmp

## Exercise 1

Create, visualize and compare (where are they the same? where are they different?) RGB composite images using: 

- Lexicographic basis: 
    * R = HH
    * G = HV
    * B = VV 
- Pauli basis: 
    * R = HH-VV
    * G = 2 * HV (or HV + VH)
    * B = HH+VV

Tips:
- include multi-look - to be be performed by averaging intensities (abs^2) on a N x N pixels moving window in range - azimuth,
- focus on a azimuth - range block within pixels [2500, 16000] and [0, 2000], respectively.


**A. Input parameters**

In [ ]:
# path the data
path = '/projects/data/polsar/'
# number of looks
looksa = 5
looksr = 5

**B. Open images, multi-look and visualize**

In [ ]:
# load SLCs
slchh = rrat(path + 'slc_16afrisr0107_Lhh_tcal_test.rat', block = [2500, 16000, 0, 2000])
slchv = rrat(path + 'slc_16afrisr0107_Lhv_tcal_test.rat', block = [2500, 16000, 0, 2000])
slcvv = rrat(path + 'slc_16afrisr0107_Lvv_tcal_test.rat', block = [2500, 16000, 0, 2000])


In [ ]:
# multi-look on HH
intensity_hh = (np.abs(slchh))**2
amphh = np.sqrt(uniform_filter(intensity_hh, [looksa, looksr]))

In [ ]:
# delete unused variables
del intensity_hh

In [ ]:
# plot both images: with and without multilooking

plt.figure(figsize =(10,10))

ax = plt.subplot(2,1,1)
plt.imshow(np.transpose(np.abs(slchh)), vmin = 0, vmax = 3*np.mean(np.abs(slchh)), cmap = 'gray', aspect = 'auto')
ax.set_title('Single look HH')
plt.tight_layout()

ax = plt.subplot(2,1,2)
plt.imshow(np.transpose(amphh), vmin = 0, vmax = 3*np.mean(np.abs(slchh)), cmap = 'gray', aspect = 'auto')
ax.set_title('Multi look HH')
plt.tight_layout()

**C. Lexicographic and Pauli basis representation**

In [ ]:
# 1. Representation in the lexicographic basis

In [ ]:
# multi-look each SLC (HH, VV, HV)
intensity_vv = (np.abs(slcvv))**2
ampvv = np.sqrt(uniform_filter(intensity_vv, [looksa, looksr]))
intensity_hv = (np.abs(slchv))**2
amphv = np.sqrt(uniform_filter(intensity_hv, [looksa, looksr]))

del intensity_vv, intensity_hv

In [ ]:
# define the 3D array for the lexicographic representation
dimaz = slchh.shape[0]
dimrg = slchh.shape[1]
print(dimaz, dimrg)
rgb_lex = np.zeros((dimrg, dimaz, 3), 'float32')

In [ ]:
# fill the array, clipping the values between 0 and 2.5xmean(SLC amplitude)
rgb_lex[:,:,0] = np.clip(np.transpose(amphh), 0,2.5*np.mean(amphh)) # R : HH amplitude
rgb_lex[:,:,1] = np.clip(np.transpose(amphv), 0,2.5*np.mean(amphv)) # G : HV amplitude
rgb_lex[:,:,2] = np.clip(np.transpose(ampvv), 0,2.5*np.mean(ampvv)) # B : VV amplitude

# normalisation: values between 0 and 1
rgb_lex[:,:,0] = rgb_lex[:,:,0] / np.max(rgb_lex[:,:,0])
rgb_lex[:,:,1] = rgb_lex[:,:,1] / np.max(rgb_lex[:,:,1])
rgb_lex[:,:,2] = rgb_lex[:,:,2] / np.max(rgb_lex[:,:,2])

In [ ]:
# 2. Representation in the Pauli basis

In [ ]:
# define and multi-look each Pauli component
pauli1 = (np.abs(slchh + slcvv))**2
pauli1 = np.sqrt(uniform_filter(pauli1, [looksa, looksr]))
pauli2 = (np.abs(slchh - slcvv))**2
pauli2 = np.sqrt(uniform_filter(pauli2, [looksa, looksr]))
pauli3 = (np.abs(2*slchv))**2
pauli3 = np.sqrt(uniform_filter(pauli3, [looksa, looksr]))

In [ ]:
# define the 3D array for the Pauli representation
rgb_pauli = np.zeros((dimrg, dimaz, 3), 'float32')

In [ ]:
# fill the array, clipping the values between 0 and 2.5xmean(amplitude)
rgb_pauli[:,:,0] = np.clip(np.transpose(pauli2), 0,2.5*np.mean(pauli2)) # R : HH-VV 
rgb_pauli[:,:,1] = np.clip(np.transpose(pauli3), 0,2.5*np.mean(pauli3)) # G : HV
rgb_pauli[:,:,2] = np.clip(np.transpose(pauli1), 0,2.5*np.mean(pauli1)) # B : HH+VV
# normalisation: values between 0 and 1
rgb_pauli[:,:,0] = rgb_pauli[:,:,0] / np.max(rgb_pauli[:,:,0])
rgb_pauli[:,:,1] = rgb_pauli[:,:,1] / np.max(rgb_pauli[:,:,1])
rgb_pauli[:,:,2] = rgb_pauli[:,:,2] / np.max(rgb_pauli[:,:,2])

In [ ]:
# 3. Plot composites in both bases

plt.figure(figsize =(10,10))

ax = plt.subplot(2,1,1)
plt.imshow(rgb_lex, aspect = 'auto')
ax.set_title('Lexicographic representation')
plt.tight_layout()

ax = plt.subplot(2,1,2)
plt.imshow(rgb_pauli, aspect = 'auto')
ax.set_title('Pauli representation')
plt.tight_layout()

## Exercise 2

Calculate and visualize the alpha angle.

**A. Compute and visualize Alpha**

In [ ]:
# Compute the alpha angle

# -- calculate the length of the Pauli vector
pauli_le = np.sqrt(np.abs(pauli1)**2 + np.abs(pauli2)**2 + np.abs(pauli3)**2)

# -- calculate the alpha angle
alpha = np.arccos(np.abs(pauli1) / pauli_le) # [rad]

In [ ]:
# Plot Pauli and alpha

plt.figure(figsize =(10,10))

ax = plt.subplot(2,1,1)
plt.imshow(rgb_pauli, aspect = 'auto')
ax.set_title('Pauli representation')
plt.colorbar()
plt.tight_layout()

ax = plt.subplot(2,1,2)
plt.imshow(np.transpose(np.degrees(alpha)), cmap = 'turbo', vmin = 0, vmax = 90 , aspect = 'auto')
ax.set_title('Alpha angle [deg]')
plt.colorbar()
plt.tight_layout()

**B. Compute and visualize Alpha and Intensity**

We will generate an HSV composite with: 
- Hue (color): alpha angle
- Saturation : constant, set to 1
- Value (intensity): total amplitude

In [ ]:
# Normalizing the alpha angle
alpha_n = alpha / (np.pi / 2)

# Build an array containing ones only
saturation = np.ones_like(alpha)

# Clip and normalize the value: total amplitude 
amp = pauli_le
amp = np.clip(amp, 0, 2.5*np.mean(amp))
amp = amp / np.max(amp)

# define a colormap
colormap = plt.colormaps.get('turbo')

# Transfer to HSV
rgb_comp = HSV_colormap_to_rgb(colormap, alpha_n, saturation, amp)

In [ ]:
del saturation, amp

In [ ]:
# Plot Pauli and the new RGB composite

plt.figure(figsize =(10,10))

ax = plt.subplot(2,1,1)
plt.imshow(rgb_pauli, aspect = 'auto')
ax.set_title('Pauli representation')
plt.tight_layout()

ax = plt.subplot(2,1,2)
plt.imshow(np.transpose(rgb_comp, axes = (1,0,2)), aspect = 'auto')
ax.set_title('HSV composite of Alpha angle [deg] and total power')
plt.tight_layout()
